In [1]:
from datetime import datetime
import requests, bs4
import re
import pandas as pd
import os
import numpy as np

Need transform functions to: 

- take in scraped scores by quarter and create individual dataframes per team like the old scraper
- record team and opponent box score stats
- Scrape four factor stats

In [54]:
url = 'https://www.basketball-reference.com/boxscores/201910280DET.html'
res = requests.get(url, 'html_parser')
res.raise_for_status()
soup = bs4.BeautifulSoup(re.sub("<!--|-->", "", res.text), 'lxml')

In [55]:
path = os.getcwd()
# os.chdir(path+'/Stat Sheets')

In [59]:
q1_box = soup.find('table',{'id':'box-DET-game-basic'})
four_fac = soup.find('table',{'id':'four_factors'})

In [60]:
four_fac

<table class="suppress_all sortable stats_table" data-cols-to-freeze="1" id="four_factors"><caption>Four Factors Table</caption>
<tr class="over_header thead">
<th aria-label="" class="over_header center" colspan="2" data-stat=""></th>
<th aria-label="" class="over_header center" colspan="4" data-stat="header_tmp">Four Factors</th><th></th>
</tr>
<tr class="thead">
<th aria-label="Team" class="sort_default_asc left" data-stat="team_id" data-tip="Team"> </th>
<th aria-label="Pace Factor" class="right" data-stat="pace" data-tip="&lt;b&gt;Pace Factor&lt;/b&gt;: An estimate of possessions per 48 minutes">Pace</th>
<th aria-label="Effective Field Goal Percentage" class="right" data-over-header="Four Factors" data-stat="efg_pct" data-tip="&lt;strong&gt;Effective Field Goal Percentage&lt;/strong&gt;&lt;br&gt;This statistic adjusts for the fact that a 3-point field goal is worth one more point than a 2-point field goal.">eFG%</th>
<th aria-label="Turnover Percentage" class="sort_default_asc ri

In [61]:
q1_df = pd.read_html(str(q1_box),flavor='bs4')[0]
four_fac_df = pd.read_html(str(four_fac), flavor='bs4')[0]

In [89]:
four_fac_df['Unnamed: 6_level_0']['ORtg'].iloc[0]

101.0

In [31]:
players = q1_df['Unnamed: 0_level_0'].drop([5],axis=0).reset_index(drop=True)

In [32]:
players = players[:-1]

In [33]:
players.rename({'Starters':'players'}, axis=1)

,players
0,Jayson Tatum
1,Gordon Hayward
2,Kemba Walker
3,Enes Kanter
4,Jaylen Brown
5,Marcus Smart
6,Carsen Edwards
7,Robert Williams
8,Daniel Theis
9,Semi Ojeleye


In [34]:
team_q1 = q1_df['Basic Box Score Stats'].drop([5], axis =0).reset_index(drop=True)

In [35]:
team_q1 = team_q1[-1:]

In [36]:
team_q1

,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
13,240,33,90,.367,7,26,.269,20,34,.588,10,31,41,18,4,2,10,29,93,NaN


In [53]:
df = pd.read_csv('2020_line_scores.csv',index_col=0)

In [54]:
df.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,home_q2,home_q3,home_q4,home_OT,home_final
0,2019-10-22,NOP,30.0,31.0,25.0,31.0,5.0,122.0,TOR,27.0,29.0,32.0,29.0,13.0,130.0
1,2019-10-22,LAL,25.0,29.0,31.0,17.0,0.0,102.0,LAC,22.0,40.0,23.0,27.0,0.0,112.0
2,2019-10-23,DET,27.0,27.0,29.0,36.0,0.0,119.0,IND,24.0,31.0,31.0,24.0,0.0,110.0
3,2019-10-23,SAC,29.0,30.0,17.0,19.0,0.0,95.0,PHO,25.0,29.0,32.0,38.0,0.0,124.0
4,2019-10-23,CLE,24.0,17.0,24.0,20.0,0.0,85.0,ORL,28.0,27.0,16.0,23.0,0.0,94.0


In [55]:
rundf = df[np.logical_or(df['home_team']=='BOS', df['away_team']=='BOS')].copy()

In [56]:
rundf.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,home_q2,home_q3,home_q4,home_OT,home_final
6,2019-10-23,BOS,25.0,23.0,20.0,25.0,0.0,93.0,PHI,20.0,29.0,28.0,30.0,0.0,107.0
16,2019-10-25,TOR,21.0,28.0,33.0,24.0,0.0,106.0,BOS,22.0,28.0,26.0,36.0,0.0,112.0
31,2019-10-26,BOS,22.0,24.0,36.0,36.0,0.0,118.0,NYK,24.0,27.0,25.0,19.0,0.0,95.0
64,2019-10-30,MIL,34.0,24.0,18.0,29.0,0.0,105.0,BOS,19.0,23.0,38.0,36.0,0.0,116.0
75,2019-11-01,NYK,23.0,27.0,24.0,28.0,0.0,102.0,BOS,22.0,27.0,26.0,29.0,0.0,104.0


In [57]:
rundf['team'] = 'BOS'
rundf['opponent'] = np.nan
rundf['home_game'] = np.nan
rundf['team_q1'] = np.nan
rundf['team_q2'] = np.nan
rundf['team_q3'] = np.nan
rundf['team_q4'] = np.nan
rundf['team_OT'] = np.nan
rundf['team_final']=np.nan
rundf['opponent_q1'] = np.nan
rundf['opponent_q1'] = np.nan
rundf['opponent_q2'] = np.nan
rundf['opponent_q3'] = np.nan
rundf['opponent_q4'] = np.nan
rundf['opponent_OT'] = np.nan
rundf['opponent_final']=np.nan

In [58]:
rundf.head()

,date,away_team,away_q1,away_q2,away_q3,away_q4,away_OT,away_final,home_team,home_q1,...,team_q3,team_q4,team_OT,team_final,opponent_q1,opponent_q2,opponent_q3,opponent_q4,opponent_OT,opponent_final
6,2019-10-23,BOS,25.0,23.0,20.0,25.0,0.0,93.0,PHI,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,2019-10-25,TOR,21.0,28.0,33.0,24.0,0.0,106.0,BOS,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,2019-10-26,BOS,22.0,24.0,36.0,36.0,0.0,118.0,NYK,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2019-10-30,MIL,34.0,24.0,18.0,29.0,0.0,105.0,BOS,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2019-11-01,NYK,23.0,27.0,24.0,28.0,0.0,102.0,BOS,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
for i in range(len(rundf)):
    
    if rundf['away_team'].iloc[i] == 'BOS':
        rundf['home_game'].iloc[i] = 0
        rundf['opponent'].iloc[i] = rundf['home_team'].iloc[i]
        rundf['team_q1'].iloc[i] = rundf['away_q1'].iloc[i]
        rundf['team_q2'].iloc[i] = rundf['away_q2'].iloc[i]
        rundf['team_q3'].iloc[i] = rundf['away_q3'].iloc[i]
        rundf['team_q4'].iloc[i] = rundf['away_q4'].iloc[i]
        rundf['team_OT'].iloc[i] = rundf['away_OT'].iloc[i]
        rundf['team_final'].iloc[i]= rundf['away_final'].iloc[i]
        rundf['opponent_q1'].iloc[i] = rundf['home_q1'].iloc[i]
        rundf['opponent_q2'].iloc[i] = rundf['home_q2'].iloc[i]
        rundf['opponent_q3'].iloc[i] = rundf['home_q3'].iloc[i]
        rundf['opponent_q4'].iloc[i] = rundf['home_q4'].iloc[i]
        rundf['opponent_OT'].iloc[i] = rundf['home_OT'].iloc[i]
        rundf['opponent_final'].iloc[i] = rundf['home_final'].iloc[i]

    else:
        rundf['home_game'].iloc[i] = 1
        rundf['opponent'].iloc[i] = rundf['away_team'].iloc[i]
        rundf['team_q1'].iloc[i] = rundf['home_q1'].iloc[i]
        rundf['team_q2'].iloc[i] = rundf['home_q2'].iloc[i]
        rundf['team_q3'].iloc[i] = rundf['home_q3'].iloc[i]
        rundf['team_q4'].iloc[i] = rundf['home_q4'].iloc[i]
        rundf['team_OT'].iloc[i] = rundf['home_OT'].iloc[i]
        rundf['team_final'].iloc[i] = rundf['home_final'].iloc[i]
        rundf['opponent_q1'].iloc[i] = rundf['away_q1'].iloc[i]
        rundf['opponent_q2'].iloc[i] = rundf['away_q2'].iloc[i]
        rundf['opponent_q3'].iloc[i] = rundf['away_q3'].iloc[i]
        rundf['opponent_q4'].iloc[i] = rundf['away_q4'].iloc[i]
        rundf['opponent_OT'].iloc[i] = rundf['away_OT'].iloc[i]
        rundf['opponent_final'].iloc[i] = rundf['away_final'].iloc[i]

/Users/ahelgeso/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [61]:
df.columns

Index(['date', 'away_team', 'away_q1', 'away_q2', 'away_q3', 'away_q4',
       'away_OT', 'away_final', 'home_team', 'home_q1', 'home_q2', 'home_q3',
       'home_q4', 'home_OT', 'home_final'],
      dtype='object')

In [62]:
cols_to_drop = ['away_team', 'away_q1', 'away_q2', 'away_q3', 'away_q4',
       'away_OT', 'away_final', 'home_team', 'home_q1', 'home_q2', 'home_q3',
       'home_q4', 'home_OT', 'home_final']

In [64]:
rundf.drop(cols_to_drop, axis=1)

,date,team,opponent,home_game,team_q1,team_q2,team_q3,team_q4,team_OT,team_final,opponent_q1,opponent_q2,opponent_q3,opponent_q4,opponent_OT,opponent_final
6,2019-10-23,BOS,PHI,0.0,25.0,23.0,20.0,25.0,0.0,93.0,20.0,29.0,28.0,30.0,0.0,107.0
16,2019-10-25,BOS,TOR,1.0,22.0,28.0,26.0,36.0,0.0,112.0,21.0,28.0,33.0,24.0,0.0,106.0
31,2019-10-26,BOS,NYK,0.0,22.0,24.0,36.0,36.0,0.0,118.0,24.0,27.0,25.0,19.0,0.0,95.0
64,2019-10-30,BOS,MIL,1.0,19.0,23.0,38.0,36.0,0.0,116.0,34.0,24.0,18.0,29.0,0.0,105.0
75,2019-11-01,BOS,NYK,1.0,22.0,27.0,26.0,29.0,0.0,104.0,23.0,27.0,24.0,28.0,0.0,102.0
96,2019-11-05,BOS,CLE,0.0,32.0,29.0,28.0,30.0,0.0,119.0,24.0,28.0,31.0,30.0,0.0,113.0
111,2019-11-07,BOS,CHO,0.0,30.0,22.0,33.0,23.0,0.0,108.0,20.0,23.0,24.0,20.0,0.0,87.0
128,2019-11-09,BOS,SAS,0.0,39.0,33.0,32.0,31.0,0.0,135.0,30.0,24.0,37.0,24.0,0.0,115.0
144,2019-11-11,BOS,DAL,1.0,25.0,29.0,30.0,32.0,0.0,116.0,21.0,33.0,25.0,27.0,0.0,106.0
154,2019-11-13,BOS,WAS,1.0,34.0,34.0,34.0,38.0,0.0,140.0,34.0,32.0,30.0,37.0,0.0,133.0
